In [2]:
from genetic_algorithm import *
import re

## hexadecimal generator

In [3]:
nr_of_input = 3
nr_of_actions = 3
nr_of_inner = 2
nr_of_genes = 8
nr_individuals = 4

res = generate_initial_genomes_for_population(nr_individuals, nr_of_genes, nr_of_input, nr_of_actions, nr_of_inner)

## Sumuj duble

In [4]:
dic = {}
for nr in res:
    dic[nr] = {}
    for i in res[nr]:
        total = dic.get(i.differ_neuron, 0) + i.weight
        n_output = f'{i.output_id}{i.output_type}'
        n_input = f'{i.input_id}{i.input_type}'
        dic[nr][i.differ_neuron] = [n_input, n_output, total]

In [5]:
dic[0]

{'0in0out': ['0in', '0out', 3.0150465435678315],
 '2in1out': ['2in', '1out', 3.404120250267053],
 '1in1out': ['1in', '1out', -3.155562337860522],
 '1in1mid': ['1in', '1mid', -1.953425911796124],
 '1mid0out': ['1mid', '0out', -3.4273157332519455],
 '1in2out': ['1in', '2out', -0.3454906149855028],
 '2in1mid': ['2in', '1mid', -2.1621852586601555]}

## brain generator

In [11]:
dic[0]

{'0in0out': ['0in', '0out', 3.0150465435678315],
 '2in1out': ['2in', '1out', 3.404120250267053],
 '1in1out': ['1in', '1out', -3.155562337860522],
 '1in1mid': ['1in', '1mid', -1.953425911796124],
 '1mid0out': ['1mid', '0out', -3.4273157332519455],
 '1in2out': ['1in', '2out', -0.3454906149855028],
 '2in1mid': ['2in', '1mid', -2.1621852586601555]}

In [17]:
brain = {}
brain.update({dic[0][i][1]:{}}) for i in dic[0] if 'out' in dic[0][i][1]
#         brain.update({dic[0][i][1]:{}})

SyntaxError: invalid syntax (Temp/ipykernel_2724/725247609.py, line 2)

In [ ]:
brain = {f'{n}out':{} for n in range(n_out+1)}

def mid_neuron(brain, edges):
    for key inbrain:
        for item in edges:
            if key == item[1] and 'mid' in item[0] and key != item[0]:
                brain[key].update({item[0]:{}})
                mid_neuron(brain[key], edges)
                
            elif key == item[1] and 'mid' in item[0] and key == item[0]:
                brain[key].update({item[0]:{}})
                
            elif key == item[1] and 'mid' not in item[0]:
                brain[key].update({item[0]:{}})

mid_neuron(brain, edges)

out